In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
import time
from time import sleep

In [4]:
air_df = pd.read_csv('./data/original_data/purpleair_original.csv')
air_df.head(1)

,id,lat,lon,name,location_type,pm_2.5,temp_f,temp_c,humidity,pressure,...,flagged,downgraded,age,is_owner,10min_avg,30min_avg,1hour_avg,6hour_avg,1day_avg,1week_avg
0,25999,30.053808,-95.494643,Villages of Bridgestone AQI,outside,1.0,50.0,10.0,62.0,1022.37,...,False,False,0,False,2.47,4.19,8.1,19.66,17.15,15.22


In [18]:
# Function to pull 2010 Census FIPs code from latitude & longitude 

def lat_lon_query(lat, lon, cen_yr):

    url_fips = f'https://geo.fcc.gov/api/census/block/find?latitude={lat}&longitude={lon}&censusYear={cen_yr}&showall=true&format=json'
    response = requests.get(url_fips)

    if response.status_code == 200:
        aq_fips = response.json()
        aq_fips_code = aq_fips['Block']['FIPS']
        # print(aq_fips_code)
    else:
        print('Error!')

    return aq_fips_code

# API: https://geo.fcc.gov/api/census/#!/block/get_block_find

**Warning:** Long run time (1 hour +)

In [95]:
# Pulling FIPs codes for the observations in our purpleair data set, using the above function

fips = air_df.apply(lambda row: lat_lon_query(row['lat'], row['lon'], 2010), axis=1)

In [96]:
# Saving FIPs codes to new column in a dataframe
air_df['fips_id'] = fips

In [97]:
# Saving the updated dataset with the fips_id column to a new CSV
air_df.to_csv('./data/original_data/purpleair_fips.csv')

In [5]:
# Pulling the new CSV into a new dataframe
pa_fips_df = pd.read_csv('./data/original_data/purpleair_fips.csv', index_col = 0)
pa_fips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14163 entries, 0 to 14162
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             14163 non-null  int64  
 1   lat            14163 non-null  float64
 2   lon            14163 non-null  float64
 3   name           14163 non-null  object 
 4   location_type  14135 non-null  object 
 5   pm_2.5         14163 non-null  float64
 6   temp_f         14163 non-null  float64
 7   temp_c         14163 non-null  float64
 8   humidity       14163 non-null  float64
 9   pressure       14163 non-null  float64
 10  last_seen      14163 non-null  object 
 11  model          14163 non-null  object 
 12  hidden         14163 non-null  bool   
 13  flagged        14163 non-null  bool   
 14  downgraded     14163 non-null  bool   
 15  age            14163 non-null  int64  
 16  is_owner       14163 non-null  bool   
 17  10min_avg      14163 non-null  float64
 18  30min_

In [6]:
# Drop null fips_ids
pa_fips_df.dropna(subset=['fips_id'], inplace=True)

In [7]:
pa_fips_df['fips_id'].astype(np.int64)[:5]

0    482015549031070
1    530530703082014
2    530530703082005
3    421298086003016
4     60190050003006
Name: fips_id, dtype: int64

In [8]:
# FIPs ID from float to int to string to fill to 15 numbers to first 12 numbers
pa_fips_df['fips_id'] = pa_fips_df['fips_id'].astype(np.int64).astype(str).str.zfill(15).apply(lambda num: num[:12])
pa_fips_df['fips_id'][:5]

0    482015549031
1    530530703082
2    530530703082
3    421298086003
4    060190050003
Name: fips_id, dtype: object

In [9]:
pa_fips_df.head(1)

,id,lat,lon,name,location_type,pm_2.5,temp_f,temp_c,humidity,pressure,...,downgraded,age,is_owner,10min_avg,30min_avg,1hour_avg,6hour_avg,1day_avg,1week_avg,fips_id
0,25999,30.053808,-95.494643,Villages of Bridgestone AQI,outside,1.0,50.0,10.0,62.0,1022.37,...,False,0,False,2.47,4.19,8.1,19.66,17.15,15.22,482015549031


### All locations  
Warning - will take ~1 minute to load the dataset below

In [10]:
# Looking at the Walkability dataset for USA...
url = 'https://edg.epa.gov/EPADataCommons/public/OA/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv'
walk_df = pd.read_csv(url)
walk_df.head()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,297836.0831
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,484945.1466
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,106705.9281
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,481828.4303
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,687684.7752


In [11]:
walk_df.dropna(subset=['GEOID10'], inplace=True)

In [12]:
# FIPs ID from float to int to string to fill to 12 numbers
walk_df['GEOID10'] = walk_df['GEOID10'].astype(np.int64).astype(str).str.zfill(12)

In [13]:
# Renaming column so we can merge on
walk_df.rename(columns={'GEOID10': 'fips_id'}, inplace=True)

In [14]:
walk_df.head(1)

,OBJECTID,fips_id,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,481130000000,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.0,3110.36082,297836.0831


In [15]:
# Merging Purple Air AQ data with Walkability for USA
air_walk = pd.merge(pa_fips_df, walk_df, on='fips_id')
air_walk.shape

(9538, 140)

In [16]:
air_walk.groupby('CSA_Name').count()['CSA'].sort_values(ascending=False)

CSA_Name
San Jose-San Francisco-Oakland, CA    5720
Los Angeles-Long Beach, CA            1033
Sacramento-Roseville, CA               865
Denver-Aurora, CO                      272
Fresno-Madera-Hanford, CA              203
Redding-Red Bluff, CA                   74
Tucson-Nogales, AZ                      44
Edwards-Glenwood Springs, CO            37
Phoenix-Mesa, AZ                        36
New York-Newark, NY-NJ-CT-PA            26
Steamboat Springs-Craig, CO             12
Hartford-East Hartford, CT              11
Birmingham-Hoover-Talladega, AL          6
Huntsville-Decatur, AL                   5
Pueblo-Cañon City, CO                    4
Little Rock-North Little Rock, AR        4
Mobile-Daphne-Fairhope, AL               1
Name: CSA, dtype: int64

In [ ]:
air_walk.to_csv('original_data/air_walk_merged.csv', index=False)

### Los Angeles Data

In [18]:
walk_la_df = walk_df.query('(STATEFP == 6) and (COUNTYFP == 37)')
walk_la_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6425 entries, 31470 to 54651
Columns: 117 entries, OBJECTID to Shape_Area
dtypes: float64(72), int64(42), object(3)
memory usage: 5.8+ MB


In [ ]:
walk_la_df.to_csv('./data/LA_data/walkability_LA.csv', index=False)

In [19]:
# Looking at the Walkability dataset for LA...
walk_la_df.head(1)

,OBJECTID,fips_id,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
31470,31471,060371872002,6.037187e+10,6,37,187200,2,348.0,"Los Angeles-Long Beach, CA",31080.0,...,0.309765,0.000133,0.17882,15,7,16,13,13.333333,3796.740538,552006.8969


In [21]:
walk_la_df['fips_id'][:5]

31470    060371872002
31471    060371873001
31472    060371873002
31476    060375437024
31477    060375437025
Name: fips_id, dtype: object

In [26]:
pa_fips_df['fips_id'][:5]

0    482015549031
1    530530703082
2    530530703082
3    421298086003
4    060190050003
Name: fips_id, dtype: object

In [27]:
air_walk_la = pd.merge(pa_fips_df, walk_la_df, on='fips_id')
air_walk_la.shape

(612, 140)

In [28]:
air_walk_la['fips_id'].value_counts().count()

496

In [29]:
air_walk_la.head()

,id,lat,lon,name,location_type,pm_2.5,temp_f,temp_c,humidity,pressure,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,47959,33.767104,-118.198957,100 Oceangate,inside,0.82,86.0,30.000000,17.0,1012.79,...,0.104685,0.000111,0.149839,18,17,20,20,19.166667,6972.639103,1.801666e+06
1,94345,34.138132,-118.009183,1230 S Mayflower (Inside),inside,0.00,94.0,34.444444,11.0,1002.99,...,0.099344,0.000064,0.085859,19,18,19,20,19.166667,3024.620797,4.501286e+05
2,95367,34.138150,-118.009264,1230 S Mayflower (Outside),outside,0.30,103.0,39.444444,3.0,1003.45,...,0.099344,0.000064,0.085859,19,18,19,20,19.166667,3024.620797,4.501286e+05
3,129297,34.019778,-118.295592,1234W 37th PL,inside,19.43,94.0,34.444444,18.0,1012.53,...,0.763976,0.000355,0.477984,6,2,18,20,14.000000,2453.349539,2.802423e+05
4,3479,34.046755,-118.155307,1381 Rock Haven St,outside,0.04,124.0,51.111111,2.0,1001.91,...,0.123065,0.000066,0.088451,19,17,13,20,17.000000,3474.082108,5.913926e+05


In [64]:
air_walk_la.to_csv('./data/LA_data/air_walk_merged_LA.csv', index = False)